# Genome Detective XML parser script

Author: Sam Nooij  
Date: 2018-05-18

Input: results.xml file as generated by [Genome Detective](http://www.genomedetective.com/app/typingtool/virus/)

Output: table in csv format, providing:

| run_id | sample_id | total_reads | low_quality_reads | non_viral_reads | viral_reads | runtime | 
| ------ | --------- | ----------- | ----------------- | --------------- | ----------- | ------ |
| ...    | ...       | ...         | ...               | ...             | ...         | ...    |

Required python packages:
 - lxml
 - icu
 - pandas
 
For automatic use in snakemake. The corresponding snakemake rule should provide the input:
 - a list of XML files (their names, as strings; e.g. [ "1_a_results.xml", "1_b_results.xml" ]
 - a name for the output (e.g. "tmp/GenomeDetective_results.csv")

In [14]:
#Import requires python libraries
from lxml import etree      #XML parser
import pandas as pd         #dataframe and csv export
from collections import defaultdict

#XML_FILES = snakemake.input[0]
#OUTPUT_FILE = snakemake.output[0]

In [4]:
### code block for developing/debugging only! ###
import glob
XML_FILES = glob.glob("../results/*_results.xml")
print(XML_FILES)

['../results/4_D_results.xml', '../results/5_5061600092_S2_results.xml', '../results/3_1_results.xml', '../results/6_A1_results.xml']


In [48]:
#Parser functions
def pull_sample_name(filename):
    """
    The sample and run IDs are in the filename, e.g.:
    "3_10_results.xml"
    where the 3 is the run ID, and the 10 is the sample ID
    """
    error_msg = """
Expected underscores in the filename with the sample ID, e.g.
3_1_results.xml
Please provide sample names in this format.
    """
    
    assert filename.count('_') >= 2, \
        "%s" % error_msg
        
    run_id = filename.split('/')[-1].split('_')[0]
    
    if filename.count('_') > 2:
        sample_id = '_'.join(filename.split('/')[-1].split('_')[1:-1])
    else:
        sample_id = filename.split('/')[-1].split('_')[1]
    
    return run_id, sample_id

def parse_xml(filename):
    """
    parse Genome Detective XML output files
    with help from:
    http://lxml.de/parsing.html#iterparse-and-iterwalk
    
    collects for each sample:
     - total number of reads
     - number of low quality reads (qc-removed)
     - number of viral reads (as identified by DIAMOND)
     - number of non-viral reads (as identified by DIAMOND)
     - total runtime (in milliseconds -> converted to seconds)
    """
    init = True         #for finding start_time
    finished = False    #for finding end_time
    qc1 = False         #for finding total_reads
    qc2 = False         #for filding low_quality_reads
    filtering = False   #for finding viral & non-viral reads
    viral = False       #for counting viral reads
    
    viral_reads = 0
    #sum all reads with "Viruses" in their taxonomy, starting from 0
        
    context = etree.iterparse(filename)
    
    for action, elem in context:
        #Always check for these keywords:
        if elem.tag == "init":
        #Indicates the **end** of the init block!
        # - By default, only the ends of blocks are parsed:
        #   this way I prevent reading elements twice
            init = False
            qc1 = True
        elif elem.tag == "qc1":
            qc1 = False
            qc2 = True
        elif elem.tag == "qc2":
            qc2 = False
            filtering = True
        elif elem.tag == "filtering":
            filtering = False
        elif elem.tag == "consensus-read-count":
            finished = True
        else:
            pass
        
        #Search for start_time
        if init and elem.tag == "start-time":
            start_time = int(elem.text)
            
        #Search for total_reads
        if qc1 and elem.tag == "read-count":
            total_reads = int(elem.text)
        
        #Search for high_quality_reads, to calculate the
        # low_quality_reads
        if qc2 and elem.tag == "read-count":
            high_quality_reads = int(elem.text)
        
        #Search for viral reads
        if filtering:
            #In the XML format, viruses can be recognized by
            #"Viruses" in the "ancestors" field (= higher 
            # taxonomic levels).
            if elem.tag == "ancestors":
                if "Viruses" in elem.text:
                    viral = True
                else:
                    viral = False
            elif elem.tag == "end-time":
                viral == False
            else:
                pass
            if viral:
                if elem.tag == "read-count-total":
                    viral_reads += int(elem.text)
                    #I take the sum of all reads in viral 
                    # buckets.
            else:
                pass
        
        #Search for end_time
        if finished and elem.tag == "end-time":
            end_time = int(elem.text)
    
    #Runtimes are reported in ms: convert to seconds
    runtime = ( end_time - start_time ) / 1000
    
    low_quality_reads = total_reads - high_quality_reads
    
    non_viral_reads = total_reads - low_quality_reads - viral_reads
    
    return {"total_reads" : total_reads,
            "low_quality_reads" : low_quality_reads, 
            "non_viral_reads" : non_viral_reads, 
            "viral_reads" : viral_reads, 
            "runtime" : runtime}

def aggregate_results(file_list):
    """
    Collects results for each sample/file and puts all in a
    single pandas dataframe object.
    Input: a list of files
    Output: a pandas dataframe
    """
    #this works easiest if the dictionary contains a list of
    # result for each value, e.g.:
    # {"run_id" : ["run3", "run4", "run5"]}
    #I.e. don't make every dictionary separately, but combine
    # all results into a single one!
    
    #Tip: search for "merging dictionaries while keeping the original values"
    #See: https://stackoverflow.com/questions/36180688/merging-two-dictionaries-while-keeping-the-original#36180760
    # and: https://code.tutsplus.com/tutorials/how-to-merge-two-python-dictionaries--cms-26230
    
    #Collect the results and put them in a dictionary.
    results_dict = defaultdict(list)
    
    for sample in file_list:
        print("Now analysing sample: %s" % sample)
        run_id, sample_id = pull_sample_name(sample)
        xml_data = parse_xml(sample)
        
        xml_data["run_id"] = run_id
        xml_data["sample_id"] = sample_id
        
        #append the results to the 'overall' results dict
        for key, value in xml_data.items():
            results_dict[key].append(value)

    #And convert the results to a dataframe
    results_df = pd.DataFrame(results_dict)
            
    return(results_df)

Testing files: ['../results/4_D_results.xml', '../results/5_5061600092_S2_results.xml', '../results/3_1_results.xml', '../results/6_A1_results.xml']
Now analysing sample: ../results/4_D_results.xml
Now analysing sample: ../results/5_5061600092_S2_results.xml
Now analysing sample: ../results/3_1_results.xml
Now analysing sample: ../results/6_A1_results.xml
   low_quality_reads  non_viral_reads run_id  runtime      sample_id  \
0              81660           702776      4     1958              D   
1             791514         10163182      5     7980  5061600092_S2   
2              62010           203950      3     7821              1   
3             168586           470322      6     7734             A1   

   total_reads  viral_reads  
0       871250        86814  
1     16289466      5334770  
2       849294       583334  
3      2011292      1372384  


In [51]:
#Script execution
if __name__ == "__main__":
    #Show which files are being analysed
    print("Provided files: %s\n" % 
          ([file.split('/')[-1] for file in XML_FILES]))
    
    #Parse/collect the results in a Pandas dataframe
    results_df = aggregate_results(XML_FILES)
    
    #Reorder the columns
    columns = ["run_id", "sample_id", "total_reads", 
               "low_quality_reads", "non_viral_reads",
               "viral_reads", "runtime"]
    results_df = results_df[columns]
    
    #And save it as a csv file
    results_df.to_csv("GenomeDetective_results.csv", 
                      index = False)
    #results_df.to_csv(OUTPUT_FILE, index = False)
    
    print("""\nDone!
The results have been written to: GenomeDetective_results.csv""")

Provided files: ['4_D_results.xml', '5_5061600092_S2_results.xml', '3_1_results.xml', '6_A1_results.xml']

Now analysing sample: ../results/4_D_results.xml
Now analysing sample: ../results/5_5061600092_S2_results.xml
Now analysing sample: ../results/3_1_results.xml
Now analysing sample: ../results/6_A1_results.xml

Done!
The results have been written to: GenomeDetective_results.csv
